In [1]:
import numpy as np

In [2]:
class DecisionStump:
    def __init__(self, size, tau):
        self.__size = size
        self.__tau = tau
    
    def generate_data(self):
        x = np.random.uniform(-1,1,self.__size)
        x.sort()
        y = np.zeros(self.__size)
        for i in range(self.__size):
            # target function: sign(x)
            if (x[i]>0):
                y[i] = 1
            else:
                y[i] = -1
            # flip
            if (np.random.random(1) < self.__tau):
                y[i] *= -1
        # print(x,y)
        return x, y
    
    def error_calc_stupid_method(self, x, y):
        min_ein = np.inf
        g_theta = 0
        g_s = 0
        h = np.zeros(self.__size)
        
        theta = -1
        s = 1
        cur_error = 0
        for i in range(self.__size):
            if ((x[i]-theta)>0):
                h[i] = s
            else:
                h[i] = -s
            if(h[i] != y[i]):
                cur_error += 1
        if (cur_error < min_ein):
            min_ein = cur_error
            g_theta = theta
            g_s = s
        elif ((cur_error == min_ein) & ((s+theta) < (g_s+g_theta))):
            g_theta = theta
            g_s = s
        
        s = -1
        cur_error = 0
        for i in range(self.__size):
            if ((x[i]-theta)>0):
                h[i] = s
            else:
                h[i] = -s
            if(h[i] != y[i]):
                cur_error += 1
        if (cur_error < min_ein):
            min_ein = cur_error
            g_theta = theta
            g_s = s
        elif ((cur_error == min_ein) & ((s+theta) < (g_s+g_theta))):
            g_theta = theta
            g_s = s
            
        for i in range(self.__size-1):
            theta = (x[i] + x[i+1]) / 2
            s = 1
            cur_error = 0
            for i in range(self.__size):
                if ((x[i]-theta)>0):
                    h[i] = s
                else:
                    h[i] = -s
                if(h[i] != y[i]):
                    cur_error += 1
            if (cur_error < min_ein):
                min_ein = cur_error
                g_theta = theta
                g_s = s
            elif ((cur_error == min_ein) & ((s+theta) < (g_s+g_theta))):
                g_theta = theta
                g_s = s

            s = -1
            cur_error = 0
            for i in range(self.__size):
                if ((x[i]-theta)>0):
                    h[i] = s
                else:
                    h[i] = -s
                if(h[i] != y[i]):
                    cur_error += 1
            if (cur_error < min_ein):
                min_ein = cur_error
                g_theta = theta
                g_s = s
            elif ((cur_error == min_ein) & ((s+theta) < (g_s+g_theta))):
                g_theta = theta
                g_s = s
        min_ein /= self.__size
        eout_zero = (1-g_s*(1-abs(g_theta)))/2
        eout_tau = (1-self.__tau*2)*eout_zero+self.__tau
        err = eout_tau-min_ein
        # print("g_theta:", g_theta, ", g_s:", g_s)
        # print("min_ein:", min_ein)
        # print("eout_zero:", eout_zero, ", eout_tau:", eout_tau)
        # print("err:", err)
        return err
    def error_calc_dp_attempt(self, x, y):        
        # start with theta = -1, s = 1
        g_theta = -1
        g_s = 1
        # all y=-1 is error
        cur_error = sum(y==-1)
        # for now it is the smallest
        min_ein = cur_error
        if (cur_error > (self.__size/2)):
            # means should use s = -1
            g_s = -1
            min_ein = self.__size-cur_error
            
        # other cases
        for i in range(self.__size-1):
            theta = (x[i] + x[i+1]) / 2
            # start with s = 1
            # use cur_error as the previous one's error
            # the only difference is y[i]
            if(y[i] == 1):
                cur_error += 1
            else:
                cur_error -= 1
            
            if (cur_error < min_ein):
                min_ein = cur_error
                g_theta = theta
                g_s = 1
            elif ((cur_error == min_ein) & ((1+theta) < (g_s+g_theta))):
                g_theta = theta
                g_s = 1
            
            # try s = -1
            if ((cur_error > (self.__size/2))):
                if ((self.__size-cur_error) < min_ein):
                    min_ein = self.__size-cur_error
                    g_theta = theta
                    g_s = -1
                elif (((self.__size-cur_error) == min_ein) & ((theta-1) < (g_s+g_theta))):
                    g_theta = theta
                    g_s = -1
            
        min_ein /= self.__size
        eout_zero = (1-g_s*(1-abs(g_theta)))/2
        eout_tau = (1-self.__tau*2)*eout_zero+self.__tau
        err = eout_tau-min_ein
        # print("g_theta:", g_theta, ", g_s:", g_s)
        # print("min_ein:", min_ein)
        # print("eout_zero:", eout_zero, ", eout_tau:", eout_tau)
        # print("err:", err)
        return err

### I will use two method to calculate error for double checking.

## 16

In [3]:
ds = DecisionStump(2,0)
total_error_stupid = 0
total_error_dp = 0
for _ in range(10000):
    x, y = ds.generate_data()
    total_error_stupid += ds.error_calc_stupid_method(x, y)
    total_error_dp += ds.error_calc_dp_attempt(x, y)
print(total_error_stupid/10000, "(by a naive implementation (straight forward but stupid))")
print(total_error_dp/10000, "(by a dp attempt)")

0.2925125547381981 (by a naive implementation (straight forward but stupid))
0.2925125547381981 (by a dp attempt)


## 17

In [4]:
ds = DecisionStump(20,0)
total_error_stupid = 0
total_error_dp = 0
for _ in range(10000):
    x, y = ds.generate_data()
    total_error_stupid += ds.error_calc_stupid_method(x, y)
    total_error_dp += ds.error_calc_dp_attempt(x, y)
print(total_error_stupid/10000, "(by a naive implementation (straight forward but stupid))")
print(total_error_dp/10000, "(by a dp attempt)")

0.023679686692862443 (by a naive implementation (straight forward but stupid))
0.023679686692862443 (by a dp attempt)


## 18

In [5]:
ds = DecisionStump(2,0.1)
total_error_stupid = 0
total_error_dp = 0
for _ in range(10000):
    x, y = ds.generate_data()
    total_error_stupid += ds.error_calc_stupid_method(x, y)
    total_error_dp += ds.error_calc_dp_attempt(x, y)
print(total_error_stupid/10000, "(by a naive implementation (straight forward but stupid))")
print(total_error_dp/10000, "(by a dp attempt)")

0.36517235547246246 (by a naive implementation (straight forward but stupid))
0.36517235547246246 (by a dp attempt)


## 19

In [6]:
ds = DecisionStump(20,0.1)
total_error_stupid = 0
total_error_dp = 0
for _ in range(10000):
    x, y = ds.generate_data()
    total_error_stupid += ds.error_calc_stupid_method(x, y)
    total_error_dp += ds.error_calc_dp_attempt(x, y)
print(total_error_stupid/10000, "(by a naive implementation (straight forward but stupid))")
print(total_error_dp/10000, "(by a dp attempt)")

0.05212226419020758 (by a naive implementation (straight forward but stupid))
0.05212226419020758 (by a dp attempt)


## 20

In [7]:
ds = DecisionStump(200,0.1)
total_error_stupid = 0
total_error_dp = 0
for _ in range(10000):
    x, y = ds.generate_data()
    total_error_stupid += ds.error_calc_stupid_method(x, y)
    total_error_dp += ds.error_calc_dp_attempt(x, y)
print(total_error_stupid/10000, "(by a naive implementation (straight forward but stupid))")
print(total_error_dp/10000, "(by a dp attempt)")

0.005314000381141413 (by a naive implementation (straight forward but stupid))
0.005314000381141413 (by a dp attempt)
